## import

In [1]:
from google.colab import drive
drive.mount('/content/drive')
filepath = "/content/drive/MyDrive/四下/ML/HW/hw4/"
train_img_path = filepath + "gz/train-images-idx3-ubyte.gz"
train_label_path = filepath + "gz/train-labels-idx1-ubyte.gz"

Mounted at /content/drive


In [2]:
import math
import numpy as np
import gzip, struct, binascii
import matplotlib.pyplot as plt

NUM_CLASSES = 10
IMG_ROWS = 28
IMG_COLS = 28
img_size = IMG_ROWS * IMG_COLS

## read_labels, read_images

In [3]:
def read_labels(label_path):
    with gzip.open(label_path, 'r') as f:
        magic, cnt_labels = struct.unpack(">2I", f.read(8)) # Big indian & unsigned int
        label_data = f.read() # unsigned byte (0~9)
        labels = np.frombuffer(label_data, dtype=np.uint8, count=cnt_labels) 
    f.close
    return labels


def read_images(img_path, label_path):
    # paths of train data
    labels = read_labels(label_path)
    labels = np.asarray(labels)
    
    with gzip.open(img_path, 'r') as f:
        magic, cnt_imgs, rows, cols = struct.unpack(">4I", f.read(16)) 
        img_size = rows * cols  
        
        images = [ [0] * (img_size) for i in range(cnt_imgs)]
        for i in range(cnt_imgs):
            for j in range(img_size):
                #gray = int(struct.unpack(">c", f.read(1))) # unsigned byte(0~255)
                gray = int(binascii.b2a_hex(f.read(1)), 16)
                if(gray > 127):
                    images[i][j] = 1
                else:
                    images[i][j] = 0
        images = np.reshape(images, (cnt_imgs, img_size))

    return images, labels

## assign_label, print_imagination, print_confusion

In [4]:
def assign_label(cnt_predict):
    # Marginize confusion -> P_label
    P_label = np.zeros((10, 10))
    for i in range(NUM_CLASSES):
        P_label[i, :] = cnt_predict[i, :]/np.sum(cnt_predict[i, :])
    P_label = P_label.ravel()

    class_each_label = np.full(10, -1)
    assigned_class = np.full(10, False)
    i = 0
    while i < 10:
        # assign from the maximum p_label
        tmp = np.argmax(P_label)
        if P_label[tmp] == 0:
            break

        # la: label, c: class
        la = tmp // 10
        c = tmp % 10
        if assigned_class[c] == False and class_each_label[la] == -1:
            class_each_label[la] = c
            assigned_class[c] = True
            i += 1

        # mark the entry 0 after assignment
        P_label[tmp] = 0
    return class_each_label


def print_imagination(P_class, logging, label):
    imagination = (P_class >= 0.5)*1
    for i in range(NUM_CLASSES):
        label_idx = label[i]
        print(logging.format(i))
        for row in range (IMG_ROWS):
            for col in range(IMG_COLS):
                idx = row * IMG_COLS + col
                print(imagination[label_idx][idx], end=' ')
            print()
        print()


def print_confusion(confusion, label, iter):
    error = 60000
    for i in range(NUM_CLASSES):
        tp = confusion[i, label[i]]
        fp = np.sum(confusion[i])-tp
        fn = np.sum(confusion[:,label[i]])-tp
        tn = 60000-tp-fp-fn
        error -= tp

        print("---------------------------------------------------------\n")
        print("Confusion Matrix :", i)
        print("\t\tPredict number ", i, " Predict not number ",i)
        print("Is numbr {i}\t\t{tp}\t\t{fn}".format(i=i, tp=tp, fn=fn))
        print("Isn't numbr {i}\t\t{fp}\t\t{tn}".format(i=i, fp=fp, tn=tn))
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        print("\nSensitivity (Successfully predict number ", i, " : {:7.5f}".format(sensitivity))
        print("Specificity (Successfully predict not number ", i, " : {:7.5f}\n".format(specificity))
        
    print('Total iteration to converge:', iter)
    print('Total error rate:', error/60000)

## EM class

In [5]:
class EM:
    def __init__(self, train_imgs, train_labels):
        # raw data
        self.X = train_imgs
        self.labels = train_labels
        self.cnt_imgs = train_imgs.shape[0]
        self.img_size = train_imgs.shape[1]
        
        self.P = np.random.uniform(0.0, 1.0, (NUM_CLASSES, self.img_size))
        self.P /= np.sum(self.P)

        self.lamb = np.full(NUM_CLASSES, 0.1)
        self.w = np.zeros((self.cnt_imgs, NUM_CLASSES))
      

    def set_params(self, w, P, lamb):
      self.w = w
      self.P = P
      self.lamb = lamb


    #---------------------------------------------------------
    # E-step:                                                #
    #   w : p(label|X)                                       #
    #   w = p(z=label, X) / marginal                         #
    #     = λ * Π p(x_pixel) / marginal                      #
    #---------------------------------------------------------
    def E_step(self):
        w = np.zeros((self.cnt_imgs, NUM_CLASSES))
        for i in range(self.cnt_imgs):
            for j in range(NUM_CLASSES):
                # w = λ * Π p(x_pixel)
                w[i, j] = self.lamb[j]
                for k in range (self.img_size):
                    p = self.P[j, k]
                    w[i, j] *= p if self.X[i, k] else (1 - p)
            
            # normalization 
            w_sum = np.sum(w[i, :])
            if w_sum:
                w[i, :] /= w_sum
        self.w = w
        return w

    
    #---------------------------------------------------------
    # M-step:                                                #
    #   w : p(Z|X)                                           #
    #   λ : Σw/n                                             #
    #   P = Σw * xi / Σw                                     #
    #---------------------------------------------------------
    def M_step(self):
        P = np.zeros((NUM_CLASSES, self.img_size))
        sum_wi = np.sum(self.w, axis=0)

        for i in range(NUM_CLASSES):
            for j in range(self.img_size):
                for k in range(self.cnt_imgs):
                    P[i, j] += self.w[k, i] * self.X[k, j]
                P[i, j] = (P[i, j] + 1e-8) / (sum_wi[i] + 1e-8 * 784) # img_size = 784
          
        self.P = P
        self.lamb = (sum_wi + 1e-8)/(np.sum(sum_wi) + 1e-8 * 10)
        return P, self.lamb

    
    #--------------------------------------------------------
    # The output will be like (just an example):            #
    #                                                       #
    #       class0  class1  class2  ... class9              #
    # lab 0   10     [200]    10          10                #
    # lab 1  [300]     10     10          10                #
    #  ...                                                  #
    # lab9    1        1       1         [300]              #
    #                                                       #
    #--------------------------------------------------------
    def predict_label(self):
        P_label = np.zeros(NUM_CLASSES)
        cnt_predict = np.zeros((NUM_CLASSES, NUM_CLASSES))

        for i in range(self.cnt_imgs):
            for j in range(NUM_CLASSES):
                P_label[j] = self.lamb[j]
                for k in range (self.img_size):
                    p = self.P[j, k]
                    P_label[j] *= p if self.X[i, k] else (1 - p)
            # predict += 1 based on MAP
            cnt_predict[self.labels[i], np.argmax(P_label)] += 1
        return cnt_predict


    #--------------------------------------------------------
    # The output are:                                       #
    # (1) Imagination of final prediction                   #
    # (2) confusion matrix and total error rate             #
    #--------------------------------------------------------
    def print_result(self, cnt_predict,  iter):
        label = assign_label(cnt_predict)
        print_imagination(self.P, "labeled class {}:", label)
        print_confusion(cnt_predict, label, iter)

## main function

In [7]:
train_mode = False

In [ ]:
if __name__ == '__main__':

    # 1. load data
    train_imgs, train_labels = read_images(train_img_path, train_label_path)
    EM_Model = EM(train_imgs, train_labels)

    # 2. loop
    iter = 20
    while True:
        # params
        iter += 1
        P_prev = np.copy(EM_Model.P)
        w_path = (filepath + "npy/w" + str(iter) + ".npy")
        p_path = (filepath + "npy/p" + str(iter) + ".npy")
        lamb_path = (filepath + "npy/lambda" + str(iter) + ".npy")
        
        '''
        # calculate the params : w, P , lamb
        if train_mode and iter > 8:
            w = EM_Model.E_step()
            P, lamb = EM_Model.M_step()

            np.save(w_path, w)
            np.save(p_path, P)
            np.save(lamb_path, lamb)
        
        else:
            w = np.load(w_path)
            P = np.load(p_path)
            lamb = np.load(lamb_path)

        # params
        diff = np.linalg.norm(P - P_prev)
        default_label = [i for i in range(10)]
        
        # logging
        print_imagination(P, "class {}:", default_label)
        print("No. of Iteration: ", iter, ", Difference: {}\n".format(diff))
        print("---------------------------------------------------------\n")
        
        # convergence criteria
        if iter == 20 or diff < 1e-1:
            break
      '''

    # 3. make final prediction and print the result
    # params
    w = np.load(w_path)
    P = np.load(p_path)
    lamb = np.load(lamb_path)
    
    # cnt the prediction
    EM_Model.set_params(w, P, lamb)
    cnt_predict = EM_Model.predict_label()
    
    # print final result
    EM_Model.print_result(cnt_predict, iter)